In [1]:
import os
import theano
from theano import *
import theano.tensor as tt
from theano.compile.ops import as_op
import matplotlib.pyplot as plt
from argparse import Namespace
import pandas as pd
import numpy as np
import pymc3 as pm
import argparse
import pickle
import json
import math
import sys  

sys.path.insert(0, '/Users/Yannis/code/fibe2-mini-project/models')
from HymodModel import HymodModel as HYMOD

# Import data

In [2]:
# Get current working directory and project root directory
cwd = os.getcwd()
rd = os.path.join(cwd.split('fibe2-mini-project/', 1)[0])
if not rd.endswith('fibe2-mini-project'):
    rd = os.path.join(cwd.split('fibe2-mini-project/', 1)[0],'fibe2-mini-project')

In [3]:
model0data = pd.read_csv(os.path.join(rd,'data','output','simulations','linear_reservoir_simulation_monthly.csv'))
model1data = pd.read_csv(os.path.join(rd,'data','output','simulations','nonlinear_reservoir_simulation_monthly.csv'))
model2data = pd.read_csv(os.path.join(rd,'data','output','simulations','hymod_simulation_monthly.csv'))

# Store net net_rainfall
nr = model0data['net_rainfall'].values.tolist()
et = model0data['evapotranspiration'].values.tolist()
n = len(nr)


with open(os.path.join(rd,'data','output','simulations/linear_reservoir_simulation_monthly_true_parameters.json'), 'r') as f:
    lrm_true_params = json.load(f)
lrm_true_args = Namespace(**lrm_true_params)

with open(os.path.join(rd,'data','output','simulations/nonlinear_reservoir_simulation_monthly_true_parameters.json'), 'r') as f:
    nlrm_true_params = json.load(f)
nlrm_true_args = Namespace(**nlrm_true_params)

with open(os.path.join(rd,'data','output','simulations/hymod_simulation_monthly_true_parameters.json'), 'r') as f:
    hymod_true_params = json.load(f)
hymod_true_args = Namespace(**hymod_true_params)

In [4]:
hymod = HYMOD(nr,et,hymod_true_args)

@as_op(itypes=[tt.dscalar,tt.dscalar,tt.dscalar,tt.dscalar,tt.dscalar], otypes=[tt.dmatrix])
def th_forward_model(param1,param2,param3,param4,param5):
    parameter_list = [param1,param2,param3,param4,param5]

    th_states = hymod.simulate(parameter_list,hymod_true_args)
    return th_states

In [5]:
# Path to files
HYMODtrace_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/hymod_samples_monthly_LRMdata_trace.pickle')
HYMODmodel_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/hymod_samples_monthly_LRMdata_model.pickle')

HYMODtrace_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/hymod_samples_monthly_NLRMdata_trace.pickle')
HYMODmodel_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/hymod_samples_monthly_NLRMdata_model.pickle')

HYMODtrace_HYMODdata_file = os.path.join(rd,'data','output','posterior_samples/hymod_samples_monthly_HYMODdata_trace.pickle')
HYMODmodel_HYMODdata_file = os.path.join(rd,'data','output','posterior_samples/hymod_samples_monthly_HYMODdata_model.pickle')

# Path to files
LRMtrace_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_monthly_LRMdata_trace.pickle')
LRMmodel_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_monthly_LRMdata_model.pickle')

LRMtrace_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_monthly_NLRMdata_trace.pickle')
LRMmodel_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_monthly_NLRMdata_model.pickle')

LRMtrace_HYMODdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_monthly_HYMODdata_trace.pickle')
LRMmodel_HYMODdata_file = os.path.join(rd,'data','output','posterior_samples/linear_reservoir_samples_monthly_HYMODdata_model.pickle')

# Path to files
NLRMtrace_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/nonlinear_reservoir_samples_monthly_LRMdata_trace.pickle')
NLRMmodel_LRMdata_file = os.path.join(rd,'data','output','posterior_samples/nonlinear_reservoir_samples_monthly_LRMdata_model.pickle')

NLRMtrace_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/nonlinear_reservoir_samples_monthly_NLRMdata_trace.pickle')
NLRMmodel_NLRMdata_file = os.path.join(rd,'data','output','posterior_samples/nonlinear_reservoir_samples_monthly_NLRMdata_model.pickle')

NLRMtrace_HYMODdata_file = os.path.join(rd,'data','output','posterior_samples/nonlinear_reservoir_samples_monthly_HYMODdata_trace.pickle')
NLRMmodel_HYMODdata_file = os.path.join(rd,'data','output','posterior_samples/nonlinear_reservoir_samples_monthly_HYMODdata_model.pickle')

In [6]:
# Read files
HYMODtrace_LRMdata = open(HYMODtrace_LRMdata_file,"rb")
HYMODmodel_LRMdata = open(HYMODmodel_LRMdata_file,"rb")
HYMODtrace_LRMdata = pickle.load(HYMODtrace_LRMdata)
HYMODmodel_LRMdata = pickle.load(HYMODmodel_LRMdata)

HYMODtrace_NLRMdata = open(HYMODtrace_NLRMdata_file,"rb")
HYMODmodel_NLRMdata = open(HYMODmodel_NLRMdata_file,"rb")
HYMODtrace_NLRMdata = pickle.load(HYMODtrace_NLRMdata)
HYMODmodel_NLRMdata = pickle.load(HYMODmodel_NLRMdata)

HYMODtrace_HYMODdata = open(HYMODtrace_HYMODdata_file,"rb")
HYMODmodel_HYMODdata = open(HYMODmodel_HYMODdata_file,"rb")
HYMODtrace_HYMODdata = pickle.load(HYMODtrace_HYMODdata)
HYMODmodel_HYMODdata = pickle.load(HYMODmodel_HYMODdata)


HYMODtraces = {"LRM":HYMODtrace_LRMdata,"NLRM":HYMODtrace_NLRMdata,"HYMOD":HYMODtrace_HYMODdata}
HYMODmodels = {"LRM":HYMODmodel_LRMdata,"NLRM":HYMODmodel_NLRMdata,"HYMOD":HYMODmodel_HYMODdata}

In [7]:
marginal_likelihoods = pd.DataFrame(columns = ['LRM_model', 'NLRM_model' , 'HYMOD_model'], index=['LRM_data','NLRM_data', 'HYMOD_data'])
bayes_factors = pd.DataFrame(columns = ['LRM_model', 'NLRM_model' , 'HYMOD_model'], index=['LRM_data','NLRM_data', 'HYMOD_data'])

In [8]:
marginal_likelihoods.xs('LRM_data')["HYMOD_model"] = HYMODmodel_LRMdata.marginal_likelihood
marginal_likelihoods.xs('NLRM_data')["HYMOD_model"] = HYMODmodel_NLRMdata.marginal_likelihood
marginal_likelihoods.xs('HYMOD_data')["HYMOD_model"] = HYMODmodel_HYMODdata.marginal_likelihood

In [9]:
from LinearReservoirModel import LinearReservoirModel as LRM
lrm = LRM(nr,lrm_true_args)

@as_op(itypes=[tt.dscalar], otypes=[tt.dmatrix])
def th_forward_model(param1):
    parameter_list = [param1]

    th_states = lrm.simulate(parameter_list,lrm_true_args.fatconv)
    return th_states

LRMtrace_LRMdata = open(LRMtrace_LRMdata_file,"rb")
LRMmodel_LRMdata = open(LRMmodel_LRMdata_file,"rb")
LRMtrace_LRMdata = pickle.load(LRMtrace_LRMdata)
LRMmodel_LRMdata = pickle.load(LRMmodel_LRMdata)

LRMtrace_NLRMdata = open(LRMtrace_NLRMdata_file,"rb")
LRMmodel_NLRMdata = open(LRMmodel_NLRMdata_file,"rb")
LRMtrace_NLRMdata = pickle.load(LRMtrace_NLRMdata)
LRMmodel_NLRMdata = pickle.load(LRMmodel_NLRMdata)

LRMtrace_HYMODdata = open(LRMtrace_HYMODdata_file,"rb")
LRMmodel_HYMODdata = open(LRMmodel_HYMODdata_file,"rb")
LRMtrace_HYMODdata = pickle.load(LRMtrace_HYMODdata)
LRMmodel_HYMODdata = pickle.load(LRMmodel_HYMODdata)

LRMtraces = {"LRM":LRMtrace_LRMdata,"NLRM":LRMtrace_NLRMdata,"HYMOD":LRMtrace_HYMODdata}
LRMmodels = {"LRM":LRMmodel_LRMdata,"NLRM":LRMtrace_NLRMdata,"HYMOD":LRMmodel_HYMODdata}

In [10]:
marginal_likelihoods.xs('LRM_data')["LRM_model"] = LRMmodel_LRMdata.marginal_likelihood
marginal_likelihoods.xs('NLRM_data')["LRM_model"] = LRMmodel_NLRMdata.marginal_likelihood
marginal_likelihoods.xs('HYMOD_data')["LRM_model"] = LRMmodel_HYMODdata.marginal_likelihood

In [11]:
from NonLinearReservoirModel import NonLinearReservoirModel as NLRM
# Instantiate linear reservoir statistical model
nlrm = NLRM(nr,nlrm_true_args)

@as_op(itypes=[tt.dscalar,tt.dscalar], otypes=[tt.dmatrix])
def th_forward_model(param1,param2):
    parameter_list = [param1,param2]

    th_states = nlrm.simulate(parameter_list,nlrm_true_args.fatconv)
    return th_states

NLRMtrace_LRMdata = open(NLRMtrace_LRMdata_file,"rb")
NLRMmodel_LRMdata = open(NLRMmodel_LRMdata_file,"rb")
NLRMtrace_LRMdata = pickle.load(NLRMtrace_LRMdata)
NLRMmodel_LRMdata = pickle.load(NLRMmodel_LRMdata)

NLRMtrace_NLRMdata = open(NLRMtrace_NLRMdata_file,"rb")
NLRMmodel_NLRMdata = open(NLRMmodel_NLRMdata_file,"rb")
NLRMtrace_NLRMdata = pickle.load(NLRMtrace_NLRMdata)
NLRMmodel_NLRMdata = pickle.load(NLRMmodel_NLRMdata)

NLRMtrace_HYMODdata = open(NLRMtrace_HYMODdata_file,"rb")
NLRMmodel_HYMODdata = open(NLRMmodel_HYMODdata_file,"rb")
NLRMtrace_HYMODdata = pickle.load(NLRMtrace_HYMODdata)
NLRMmodel_HYMODdata = pickle.load(NLRMmodel_HYMODdata)

NLRMtraces = {"LRM":NLRMtrace_LRMdata,"NLRM":NLRMtrace_NLRMdata,"HYMOD":NLRMtrace_HYMODdata}
NLRMmodels = {"LRM":NLRMtrace_LRMdata,"NLRM":NLRMmodel_NLRMdata,"HYMOD":NLRMmodel_HYMODdata}

In [12]:
marginal_likelihoods.xs('LRM_data')["NLRM_model"] = NLRMmodel_LRMdata.marginal_likelihood
marginal_likelihoods.xs('NLRM_data')["NLRM_model"] = NLRMmodel_NLRMdata.marginal_likelihood
marginal_likelihoods.xs('HYMOD_data')["NLRM_model"] = NLRMmodel_HYMODdata.marginal_likelihood

In [13]:
log_marginal_likelihoods = marginal_likelihoods.applymap(lambda x: np.log(x))

In [12]:
for m in ["LRM","HYMOD"]:
    for d in ["LRM","HYMOD"]:
        if m == d:
            bayes_factors.xs((d+'_data'))[(m+'_model')] = 1
        else:
            bayes_factors.xs((d+'_data'))[(m+'_model')] = log_marginal_likelihoods.xs((d+'_data'))[(m+'_model')]/log_marginal_likelihoods.xs((m+'_data'))[(d+'_model')]

In [14]:
marginal_likelihoods

,LRM_model,NLRM_model,HYMOD_model
LRM_data,2.58252e-11,1.78758e-17,6.14473e-66
NLRM_data,1.57472e-28,3.27177e-11,8.2266e-69
HYMOD_data,3.30797e-66,1.13218e-64,8.96483e-12


In [15]:
log_marginal_likelihoods

,LRM_model,NLRM_model,HYMOD_model
LRM_data,-24.379671,-38.563083,-150.155022
NLRM_data,-64.018304,-24.143104,-156.770999
HYMOD_data,-150.774281,-147.241301,-25.437712
